# Banco Mundial: préstamos a países en desarrollo

Detalles sobre la organización en: https://www.worldbank.org/en/about/what-we-do

### Disponible en Kaggle en:
https://www.kaggle.com/theworldbank/world-banks-major-contracts

El Banco Mundial es una organización internacional que se fundó en 1944 para reconstruir Europa después de la Segunda Guerra Mundial. Es uno de una variedad de organizaciones que ayudan a dar forma y equilibrar la economía mundial. Hoy, su función principal es combatir la pobreza ofreciendo asistencia para el desarrollo a países del tercer mundo de ingresos medios y bajos.
Según el propio Banco Mundial, la organización tiene dos objetivos muy específicos para 2030:
* Poner fin a la pobreza extrema disminuyendo el porcentaje de personas que viven con menos de `$`1.90 por día a no más del 3% 
* Promover la prosperidad compartida fomentando el crecimiento de los ingresos del 40% inferior para cada país.

# Variables y significado

1. As of Date: 6-Sep-17, date when the file was generated. 
2. Fiscal Year: año fiscal en el que se ha llevado a cabo el préstamo.
3. Region: área geográfica del país al que se le ha concedido el préstamo.
4. Borrower Country: país al que se le ha concedido el préstamo.
5. Borrower Country Code: código de dicho país.
6. Project ID: código interno del préstamo.
7. Project Name: nombre del proyecto que se llevará a cabo.
8. Procurement Type: área para la cual se utilizará el préstamo.
9. Procurement Category: ídem al anterior pero con una categorización más amplia.
10. Procurement Method: tipo de adjudicación.
11. Product Line: área interna a la que se le ha concedido el préstamo.
12. Major Sector: economic sector económico al que se destinará.
13. WB Contract Number: número interno de contrato.
14. Contract Description: descripción.
15. Contract Signing Date: fecha en la que el contrato se oficializó.
16. Supplier: proveedor principal que trabaja en el proyecto.
17. Supplier Country: país del proveedor principal.
18. Supplier Country Code: código del país del proveedor.
19. Total Contract Amount (USD): cantidad total en dólares.
20. Borrower Contract Reference Number: número de referencia del contrato.

In [1]:
from pyspark.sql import SparkSession
#create a SparkSession
spark = (SparkSession
    .builder
    .appName("Example-3_6")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.2")
    .getOrCreate())

# Sobre el dataset anterior (Major_Contract_Awards.csv) se pide:

(1 punto) Ejercicio 1

Leerlo tratando de que Spark infiera el tipo de dato de cada columna, y cachearlo.
Puesto que existen columnas que contienen una coma enmedio del valor, en esos casos los valores vienen entre comillas dobles. Spark ya contempla esta posibilidad y puede leerlas adecuadamente si al leer le indicamos las siguientes opciones adicionales además de las que ya sueles usar: .option("quote", "\"").option("escape", "\"").
Asegúrate de que las filas que no tienen el formato correcto sean descartadas, indicando también la opción mode con el valor DROPMALFORMED como vimos en clase.

In [2]:
from pyspark.sql import functions as F

contractsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .option("quote", "\"")\
                 .option("escape", "\"")\
                 .option("mode", "DROPMALFORMED")\
                 .csv("ModeloB Major_Contract_Awards.csv")

contractsDF.cache()

DataFrame[As of Date: string, Fiscal Year: string, Region: string, Borrower Country: string, Borrower Country Code: string, Project ID: string, Project Name: string, Procurement Type: string, Procurement Category: string, Procurement Method: string, Product line: string, Major Sector: string, WB Contract Number: int, Contract Description: string, Contract Signing Date: string, Supplier: string, Supplier Country: string, Supplier Country Code: string, Total Contract Amount (USD): string, Borrower Contract Reference Number: string]

Ejercicio 2

* La columna **Total Contract Amount (USD)** es en realidad numérica, pero todas las cantidades incluyen el signo `$` por lo que Spark la reconoce como string. Para corregir este comportamiento, vamos a eliminar el `$` de todas las filas utilizando la función `F.regexp_replace("Total Contract Amount (USD)", "\$", "")` donde `"\$"` es el string que queremos reemplazar (hay que escaparlo poniendo `\` delante porque sino el `$` se interpreta como un carácter especial), y siendo el nuevo string el string vacío, `""`. Esta función pertenece al paquete `pyspark.sql.functions`, por lo que ya funciona de manera distribuida, y devuelve como resultado un objeto columna transformado. 

* Aplica esta función dentro de la función `withColumn` para **reemplazar** la columna `Total Contract Amount (USD)` ya existente por la columna devuelta por `regexp_replace`. La manera de utilizarla es totalmente análoga a la utilización de, por ejemplo, la función `F.when` dentro de `withColumn`. 
* Aprovecha también para hacer un casting del objeto columna devuelto por regexp_replace, que es una columna de strings, a una columna de enteros: `F.regexp_replace(...).cast(...)`. Almacena el DF resultante en la variable `contractsDFenteros`, **cachéala** y utilízala a partir de este momento para trabajar en las celdas posteriores, salvo que la celda indique lo contrario.

In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

contractsDFenteros = contractsDF.withColumn("Total Contract Amount (USD)",
                                            F.regexp_replace("Total Contract Amount (USD)",
                                                             "\$",
                                                             "").cast("Integer")
                                           )
contractsDFenteros.cache()

DataFrame[As of Date: string, Fiscal Year: string, Region: string, Borrower Country: string, Borrower Country Code: string, Project ID: string, Project Name: string, Procurement Type: string, Procurement Category: string, Procurement Method: string, Product line: string, Major Sector: string, WB Contract Number: int, Contract Description: string, Contract Signing Date: string, Supplier: string, Supplier Country: string, Supplier Country Code: string, Total Contract Amount (USD): int, Borrower Contract Reference Number: string]

Ejercicio 3

Partiendo de contractsDFenteros, crear un nuevo DF donde la columna "Region" sea reemplazada por otra con mismo nombre, de tipo string en la que todos los valores de la columna original (LATIN AMERICA AND CARIBBEAN, SOUTH ASIA, OTHER ... etc) estén traducidos al español. Puedes elegir la traducción que más te guste, pero debe mantenerse el mismo número de categorías que ya había, que eran siete.
El evaluador oculto comprobará que sigue habiendo el mismo número de ejemplos en cada categoría con el nuevo nombre, y que las categorías efectivamente se han traducido (ninguna se debe llamar igual que antes). Puedes cambiar AFRICA por África.

In [12]:
contractsDFenteros.groupBy("Region").count().toPandas()

,Region,count
0,LATIN AMERICA AND CARIBBEAN,29123
1,SOUTH ASIA,18903
2,OTHER,75
3,AFRICA,39212
4,MIDDLE EAST AND NORTH AFRICA,7818
5,EAST ASIA AND PACIFIC,23404
6,EUROPE AND CENTRAL ASIA,29980


In [19]:
 contractsTranslatedDF = contractsDFenteros.withColumn("Region",
                                                         F.when(
                                                           F.col("Region") == "LATIN AMERICA AND CARIBBEAN",
                                                             "LATINOAMERICA Y CARIBE"
                                                         )\
                                                         .when(
                                                             F.col("Region") == "SOUTH ASIA",
                                                             "SUDASIA"
                                                         )\
                                                         .when(
                                                             F.col("Region") == "AFRICA",
                                                             "ÁFRICA"
                                                         )\
                                                         .when(
                                                             F.col("Region") == "MIDDLE EAST AND NORTH AFRICA",
                                                             "ESTE MEDIO Y NORTE DE AFRICA"
                                                         )\
                                                         .when(
                                                             F.col("Region") == "EAST ASIA AND PACIFIC",
                                                             "ESTE DE ASIA Y PACIFICO"
                                                         )\
                                                         .when(
                                                             F.col("Region") == "EUROPE AND CENTRAL ASIA",
                                                             "EUROPA Y ASIA CENTRAL"
                                                         )\
                                                         .when(
                                                             F.col("Region") == "OTHER",
                                                             "OTROS"
                                                         )\
                                                      )
    
contractsTranslatedDF.select("Region").toPandas()

,Region
0,ÁFRICA
1,ÁFRICA
2,ÁFRICA
3,ÁFRICA
4,ESTE DE ASIA Y PACIFICO
...,...
148510,ÁFRICA
148511,ÁFRICA
148512,ÁFRICA
148513,EUROPA Y ASIA CENTRAL


 Ejercicio 4

Partiendo de contractsTranslatedDF, crear un nuevo DataFrame de una sola fila que contenga, por este orden de columnas, el número de categorías distintas existentes en cada una de las columnas Procurement Type, Procurement Category y Procurement Method. Pista: crear cada una de estas tres columnas al vuelo con select(). Renombrar cada columna de conteo para que se llame igual que la propia columna que estamos contando.

In [29]:
numeroCategoriasDF = contractsTranslatedDF.select(
    F.countDistinct("Procurement Type").alias("Procurement Type"),
    F.countDistinct("Procurement Category").alias("Procurement Category"),
    F.countDistinct("Procurement Method").alias("Procurement Method")
)
numeroCategoriasDF.toPandas()

,Procurement Type,Procurement Category,Procurement Method
0,60,5,18


Ejercicio 5

Partiendo de contractsDFenteros definido anteriormente, crear un pipeline formado por dos etapas: un indexador de la columna categórica Procurement Method y un discretizador (bucketizer) de la columna numérica que habíamos convertido a entero al principio, Total Contract Amount (USD), de manera que sea convertida en una columna de números reales empezando en 0 y cuya parte decimal siempre sea 0.
Para el indexador, si una vez entrenado le llegasen etiquetas que no ha visto antes, deberá eliminar esas filas (recordar la opción adecuada que vimos en clase). La columna de salida debe llamarse ProcurementIndexed.
Para el discretizador, utilizar como puntos de corte los siguientes: (-Inf, 0, 100000, 200000, 300000, 400000, Inf). La columna de salida debe llamarse TotalDiscretized.
Una vez creados ambos, componerlos para crear un Pipeline, y aplicarlo a contractsDFenteros para entrenar y a continuación también para predecir (es decir, transformarlo). El DF resultante de la transformación debe almacenarse en la variable contractsTransformedDF